# Using EDSL to sense check data
This notebook provides example code for sense checking survey data using [EDSL](https://docs.expectedparrot.com), an open-source library for simulating surveys, experiments and market research with AI agents and large language models. 

## Contents
Using a set of responses to a survey about online marketplaces as an example, we demonstrate EDSL methods for: 

1. Evaluating survey questions (e.g., for clarity and improvements)
2. Analyzing each respondent's set of answers (e.g., to summarize or identify sentiment, themes, etc.)
3. Reviewing each answer individually (e.g., to evaluate its relevance or usefulness)

## Coop
We also show how to post EDSL questions, surveys, results and notebooks (like this one) to the [Coop: a new platform for creating and sharing LLM-based research](https://www.expectedparrot.com/explore). 

## How EDSL works
EDSL is a flexible library that can be used to perform a broad variety of research tasks. A typical workflow consists of the following steps:

* Construct questions in EDSL 
* Add data to the questions (e.g., for data labeling tasks)
* Use an AI agent to answer the questions
* Select a language model to generate the answers
* Analyze results in a formatted dataset

## Technical setup
Before running the code below please ensure that you have completed setup:

* [Install](https://docs.expectedparrot.com/en/latest/installation.html) the EDSL library.
* Create a [Coop account](https://www.expectedparrot.com/login) and activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) OR store your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use.

Our [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) provides examples of EDSL basic components. 

## Example data
Our example data is a CSV consisting of several questions and a few rows of responses:

In [1]:
import pandas as pd

df = pd.read_csv("marketplace_survey_results.csv")
df

,Respondent ID,What do you like most about using our online marketplace?,What is one feature you would like to see added to improve your shopping experience?,Can you describe a recent experience where you were dissatisfied with our service?,How do you feel about the current product search and filtering options?,Is there anything else you would like to share about your experience with us?
0,101,The wide variety of products and the ease of use.,It would be great to have a personalized recom...,I was disappointed when an item I ordered arri...,The search and filtering options are intuitive...,"No, keep up the great work!"
1,102,I enjoy the simplicity of the interface.,A feature that helps compare similar products ...,No complaints here.,I find the product search to be pretty effective.,I think the sky is a beautiful shade of purple...
2,103,The platform is user-friendly and offers a vas...,Would love to see an option to save and compar...,"My delivery was late by a few days, which was ...",It’s okay.,No.


## Creating questions about the data
There are many questions we might want to ask about the data, such as:

* Does this survey question have any logical or syntactical problems? {{ *question* }}
* What is the overall sentiment of this respondent's answers? {{ *responses* }}
* Is this answer responsive to the question that was asked? {{ *question* }} {{ *answer* }}

## Question types
EDSL comes with many common question types that we can select from based on the form of the response that we want to get back from the model: multiple choice, checkbox, linear scale, free text, etc. [Learn more about EDSL question types](https://docs.expectedparrot.com/en/latest/questions.html).

Here we construct `Question` objects for the questions that we want to ask about the data, using `{{ placeholders }}` for the information that we will add to the questions in the steps that follow:

In [2]:
from edsl import QuestionFreeText, QuestionMultipleChoice, QuestionYesNo

In [3]:
q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = "Describe any logical or syntactical problems in the following survey question: {{ question }}"
)



In [4]:
q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the overall sentiment of this respondent's survey answers? {{ responses }}",
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

In [5]:
q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = "Is this answer responsive to the question that was asked? Question: {{ question }} Answer: {{ answer }}"
)

## Adding survey data to the questions
Next we'll add our data to our questions. This can be done efficiently by creating a `ScenarioList` representing the data. The individual `Scenario` objects in the list can be constructed in a variety of ways depending on the information that we want to include in a particular question.

We start by calling the `from_csv()` method to create a `ScenarioList` for the data in its original form. We can see that this generates a `Scenario` dictionary for each respondent's set of answers with key/value pairs for the individual questions and answers: 

In [6]:
from edsl import ScenarioList

In [7]:
sl = ScenarioList.from_csv("marketplace_survey_results.csv")
sl

ScenarioList([Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'}), Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'}), Scenario({'Respondent ID': '103', 'What do you like most about using our online marketplace?': 'The platform is user-friendly and offers a vast selection of products.', 'What is one feature you would like to see added to improve your shopping experience?': 'Would love to see an option to save and compare different products.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'My delivery was late by a few days, which was frustrating.', 'How do you feel about the current product search and filtering options?': 'It’s okay.', 'Is there anything else you would like to share about your experience with us?': 'No.'})])

## Evaluating the questions
For our first question we want to create a `Scenario` for each survey question:

In [8]:
from edsl import QuestionFreeText, Survey

q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = "Describe any logical or syntactical problems in the following survey question: {{ question }}"
)

q_improved = QuestionFreeText(
    question_name = "improved",
    question_text = "Please draft an improved version of the survey question. Return only the revised question text."
)

survey = Survey([q_logic, q_improved]).add_targeted_memory(q_improved, q_logic)

The survey questions are the `parameters` of the `ScenarioList` created above:

In [9]:
questions = list(sl.parameters)
questions

['How do you feel about the current product search and filtering options?',
 'What do you like most about using our online marketplace?',
 'Is there anything else you would like to share about your experience with us?',
 'Can you describe a recent experience where you were dissatisfied with our service?',
 'Respondent ID',
 'What is one feature you would like to see added to improve your shopping experience?']

We can pass them to the `from_list()` method to create a new `ScenarioList`, specifying that the key for each `Scenario` will be `question` in order to match the parameter of our logic question:

In [10]:
sl_questions = ScenarioList.from_list("question", questions)
sl_questions

ScenarioList([Scenario({'question': 'How do you feel about the current product search and filtering options?'}), Scenario({'question': 'What do you like most about using our online marketplace?'}), Scenario({'question': 'Is there anything else you would like to share about your experience with us?'}), Scenario({'question': 'Can you describe a recent experience where you were dissatisfied with our service?'}), Scenario({'question': 'Respondent ID'}), Scenario({'question': 'What is one feature you would like to see added to improve your shopping experience?'})])

We add the scenarios to the survey when we run it:

In [11]:
results = survey.by(sl_questions).run()

This generates a dataset of `Results` that we can access with built-in methods for analysis:

In [12]:
results.select("question", "logic", "improved").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario                            ┃ answer                              ┃ answer                              ┃
┃ .question                           ┃ .logic                              ┃ .improved                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ How do you feel about the current   │ The survey question 'How do you     │ On a scale of 1 to 5, how satisfied │
│ product search and filtering        │ feel about the current product      │ are you with the current product    │
│ options?                            │ search and filtering options?' is   │ search functionality, and how       │
│                                     │ generally clear but could be        │ satisfied are you with the product  │
│                                     │ improved by being more specific. It │ filtering options available on our  │
│                                     │ combines two different aspects      │ website?                            │
│                                     │ (search and filtering) into one     │                                     │
│                                     │ question, which might confuse the   │                                     │
│                                     │ respondent if they have different   │                                     │
│                                     │ feelings about each aspect. To      │                                     │
│                                     │ address this, the question could be │                                     │
│                                     │ split into two separate questions   │                                     │
│                                     │ to allow for more specific feedback │                                     │
│                                     │ on each feature. Additionally, the  │                                     │
│                                     │ question is asking for a subjective │                                     │
│                                     │ feeling, which may result in varied │                                     │
│                                     │ and qualitative data that can be    │                                     │
│                                     │ difficult to quantify. Including a  │                                     │
│                                     │ scale or specific aspects to rate   │                                     │
│                                     │ could help in quantifying the       │                                     │
│                                     │ responses.                          │                                     │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│ What do you like most about using   │ The survey question seems to be     │ What features of our online         │
│ our online marketplace?             │ clear and straightforward without   │ marketplace do you find most        │
│                                     │ any apparent logical or syntactical │ beneficial? Please rank the         │
│                                     │ problems. It asks for a subjective  │ following aspects from most to      │
│                                     │ response regarding the user's       │ least preferred: ease of use,       │
│                                     │ preference about the online         │ product variety, customer service,  │
│                                     │ marketplace. However, if the survey │ pricing, and site performance.      │
│                                     │ aims to collect structured data, it │                                     │
│                                     │ might be benefic

[Learn more about working with results](https://docs.expectedparrot.com/en/latest/results.html).

## Evaluating respondents' collective answers
Next we can create a `ScenarioList` for each respondent's answers to use with our question about sentiment:

In [13]:
sl_responses = ScenarioList.from_list("responses", sl['scenarios'])
sl_responses

ScenarioList([Scenario({'responses': {'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!', 'edsl_version': '0.1.33.dev1', 'edsl_class_name': 'Scenario'}}), Scenario({'responses': {'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.', 'edsl_version': '0.1.33.dev1', 'edsl_class_name': 'Scenario'}}), Scenario({'responses': {'Respondent ID': '103', 'What do you like most about using our online marketplace?': 'The platform is user-friendly and offers a vast selection of products.', 'What is one feature you would like to see added to improve your shopping experience?': 'Would love to see an option to save and compare different products.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'My delivery was late by a few days, which was frustrating.', 'How do you feel about the current product search and filtering options?': 'It’s okay.', 'Is there anything else you would like to share about your experience with us?': 'No.', 'edsl_version': '0.1.33.dev1', 'edsl_class_name': 'Scenario'}})])

Next we add these scenarios to our sentiment question (and any others we want to add) and run it:

In [14]:
from edsl import QuestionMultipleChoice, QuestionLinearScale, Survey

q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the overall sentiment of this respondent's survey answers? {{ responses }}",
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

q_recommend = QuestionLinearScale(
    question_name = "recommend",
    question_text = "On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? {{ responses }}",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all likely", 5:"Very likely"}
)

survey = Survey([q_sentiment, q_recommend])

In [15]:
results = survey.by(sl_responses).run()

In [16]:
results.select("responses", "sentiment", "recommend").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ scenario                                                                      ┃ answer             ┃ answer     ┃
┃ .responses                                                                    ┃ .sentiment         ┃ .recommend ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ {'Respondent ID': '101', 'What do you like most about using our online        │ Somewhat satisfied │ 4          │
│ marketplace?': 'The wide variety of products and the ease of use.', 'What is  │                    │            │
│ one feature you would like to see added to improve your shopping              │                    │            │
│ experience?': 'It would be great to have a personalized recommendation system │                    │            │
│ based on my browsing history.', 'Can you describe a recent experience where   │                    │            │
│ you were dissatisfied with our service?': 'I was disappointed when an item I  │                    │            │
│ ordered arrived damaged, but customer service quickly resolved it.', 'How do  │                    │            │
│ you feel about the current product search and filtering options?': 'The       │                    │            │
│ search and filtering options are intuitive and work well for me.', 'Is there  │                    │            │
│ anything else you would like to share about your experience with us?': 'No,   │                    │            │
│ keep up the great work!', 'edsl_version': '0.1.33.dev1', 'edsl_class_name':   │                    │            │
│ 'Scenario'}                                                                   │                    │            │
├───────────────────────────────────────────────────────────────────────────────┼────────────────────┼────────────┤
│ {'Respondent ID': '102', 'What do you like most about using our online        │ Somewhat satisfied │ 4          │
│ marketplace?': 'I enjoy the simplicity of the interface.', 'What is one       │                    │            │
│ feature you would like to see added to improve your shopping experience?': 'A │                    │            │
│ feature that helps compare similar products side by side would be useful.',   │                    │            │
│ 'Can you describe a recent experience where you were dissatisfied with our    │                    │            │
│ service?': 'No complaints here.', 'How do you feel about the current product  │                    │            │
│ search and filtering options?': 'I find the product search to be pretty       │                    │            │
│ effective.', 'Is there anything else you would like to share about your       │                    │            │
│ experience with us?': 'I think the sky is a beautiful shade of purple         │                    │            │
│ today.', 'edsl_version': '0.1.33.dev1', 'edsl_class_name': 'Scenario'}        │                    │            │
├───────────────────────────────────────────────────────────────────────────────┼────────────────────┼────────────┤
│ {'Respondent ID': '103', 'What do you like most about using our online        │ Somewhat satisfied │ 3          │
│ marketplace?': 'The platform is user-friendly and offers a vast selection of  │                    │            │
│ products.', 'What is one feature you would like to see added to improve your  │                    │            │
│ shopping experience?': 'Would love to see an option to save and compare       │                    │            │
│ different products.', 'Can you describe a recent experience where you were    │                    │            │
│ dissatisfied with our service?': 'My delivery was late by a few days, which   │                    │            │
│ was frustrating.', 'How do you feel about the current 

## Evaluating individual answers
Next we create a `ScenarioList` for each individual question and answer to use with our question about the responsiveness of each answer. We can use the `unpivot()` method to expand the scenarios by desired identifiers (e.g., respondent ID):

In [17]:
sl_qa = sl.unpivot(id_vars = ["Respondent ID"])
sl_qa

ScenarioList([Scenario({'Respondent ID': '101', 'variable': 'What do you like most about using our online marketplace?', 'value': 'The wide variety of products and the ease of use.'}), Scenario({'Respondent ID': '101', 'variable': 'What is one feature you would like to see added to improve your shopping experience?', 'value': 'It would be great to have a personalized recommendation system based on my browsing history.'}), Scenario({'Respondent ID': '101', 'variable': 'Can you describe a recent experience where you were dissatisfied with our service?', 'value': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.'}), Scenario({'Respondent ID': '101', 'variable': 'How do you feel about the current product search and filtering options?', 'value': 'The search and filtering options are intuitive and work well for me.'}), Scenario({'Respondent ID': '101', 'variable': 'Is there anything else you would like to share about your experience with us?', 'value': 'No, keep up the great work!'}), Scenario({'Respondent ID': '102', 'variable': 'What do you like most about using our online marketplace?', 'value': 'I enjoy the simplicity of the interface.'}), Scenario({'Respondent ID': '102', 'variable': 'What is one feature you would like to see added to improve your shopping experience?', 'value': 'A feature that helps compare similar products side by side would be useful.'}), Scenario({'Respondent ID': '102', 'variable': 'Can you describe a recent experience where you were dissatisfied with our service?', 'value': 'No complaints here.'}), Scenario({'Respondent ID': '102', 'variable': 'How do you feel about the current product search and filtering options?', 'value': 'I find the product search to be pretty effective.'}), Scenario({'Respondent ID': '102', 'variable': 'Is there anything else you would like to share about your experience with us?', 'value': 'I think the sky is a beautiful shade of purple today.'}), Scenario({'Respondent ID': '103', 'variable': 'What do you like most about using our online marketplace?', 'value': 'The platform is user-friendly and offers a vast selection of products.'}), Scenario({'Respondent ID': '103', 'variable': 'What is one feature you would like to see added to improve your shopping experience?', 'value': 'Would love to see an option to save and compare different products.'}), Scenario({'Respondent ID': '103', 'variable': 'Can you describe a recent experience where you were dissatisfied with our service?', 'value': 'My delivery was late by a few days, which was frustrating.'}), Scenario({'Respondent ID': '103', 'variable': 'How do you feel about the current product search and filtering options?', 'value': 'It’s okay.'}), Scenario({'Respondent ID': '103', 'variable': 'Is there anything else you would like to share about your experience with us?', 'value': 'No.'})])

We can call the `rename()` method to rename the keys as desired to match our question parameters syntax:

In [18]:
sl_qa = sl_qa.rename({"Respondent ID": "id", "variable": "question", "value": "answer"})
sl_qa

ScenarioList([Scenario({'id': '101', 'question': 'What do you like most about using our online marketplace?', 'answer': 'The wide variety of products and the ease of use.'}), Scenario({'id': '101', 'question': 'What is one feature you would like to see added to improve your shopping experience?', 'answer': 'It would be great to have a personalized recommendation system based on my browsing history.'}), Scenario({'id': '101', 'question': 'Can you describe a recent experience where you were dissatisfied with our service?', 'answer': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.'}), Scenario({'id': '101', 'question': 'How do you feel about the current product search and filtering options?', 'answer': 'The search and filtering options are intuitive and work well for me.'}), Scenario({'id': '101', 'question': 'Is there anything else you would like to share about your experience with us?', 'answer': 'No, keep up the great work!'}), Scenario({'id': '102', 'question': 'What do you like most about using our online marketplace?', 'answer': 'I enjoy the simplicity of the interface.'}), Scenario({'id': '102', 'question': 'What is one feature you would like to see added to improve your shopping experience?', 'answer': 'A feature that helps compare similar products side by side would be useful.'}), Scenario({'id': '102', 'question': 'Can you describe a recent experience where you were dissatisfied with our service?', 'answer': 'No complaints here.'}), Scenario({'id': '102', 'question': 'How do you feel about the current product search and filtering options?', 'answer': 'I find the product search to be pretty effective.'}), Scenario({'id': '102', 'question': 'Is there anything else you would like to share about your experience with us?', 'answer': 'I think the sky is a beautiful shade of purple today.'}), Scenario({'id': '103', 'question': 'What do you like most about using our online marketplace?', 'answer': 'The platform is user-friendly and offers a vast selection of products.'}), Scenario({'id': '103', 'question': 'What is one feature you would like to see added to improve your shopping experience?', 'answer': 'Would love to see an option to save and compare different products.'}), Scenario({'id': '103', 'question': 'Can you describe a recent experience where you were dissatisfied with our service?', 'answer': 'My delivery was late by a few days, which was frustrating.'}), Scenario({'id': '103', 'question': 'How do you feel about the current product search and filtering options?', 'answer': 'It’s okay.'}), Scenario({'id': '103', 'question': 'Is there anything else you would like to share about your experience with us?', 'answer': 'No.'})])

In [19]:
from edsl import QuestionYesNo

q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = "Is this answer responsive to the question that was asked? Question: {{ question }} Answer: {{ answer }}"
)

In [20]:
results = q_responsive.by(sl_qa).run()

In [21]:
(results
 .filter("responsive == 'No'")
 .select("id", "question", "answer")
 .print(format="rich")
)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario ┃ scenario                                          ┃ scenario                                         ┃
┃ .id      ┃ .question                                         ┃ .answer                                          ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 102      │ Can you describe a recent experience where you    │ No complaints here.                              │
│          │ were dissatisfied with our service?               │                                                  │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ 102      │ Is there anything else you would like to share    │ I think the sky is a beautiful shade of purple   │
│          │ about your experience with us?                    │ today.                                           │
└──────────┴───────────────────────────────────────────────────┴──────────────────────────────────────────────────┘

## Uploading content to the Coop
[Coop](https://www.expectedparrot.com/explore) is a new platform for creating, storing and sharing LLM-based research. It is fully integrated with EDSL, and a convenient place to post and access surveys, agents, results and notebooks. [Learn more about using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the contents of this notebook:

In [22]:
from edsl import Notebook

In [23]:
n = Notebook(path = "scenariolist_unpivot.ipynb")

In [24]:
n.push(description = "ScenarioList methods for sense checking survey data", visibility = "public")

{'description': 'ScenarioList methods for sense checking survey data',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/a8346e52-3284-4734-9e10-8d755059192f',
 'uuid': 'a8346e52-3284-4734-9e10-8d755059192f',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [25]:
n = Notebook(path = "scenariolist_unpivot.ipynb")

n.patch(uuid = "a8346e52-3284-4734-9e10-8d755059192f", value = n)

{'status': 'success'}